In [3]:
import json
import math

# Activation functions
def relu(x):
    return max(0, x)

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def softmax(outputs):
    exp_values = [math.exp(x) for x in outputs]
    total = sum(exp_values)
    return [x / total for x in exp_values]

def tanh(x):
    return math.tanh(x)

# Function to apply activation by name
def apply_activation(activation, values):
    if activation == "softmax":
        # Apply softmax to the entire list
        return softmax(values)
    else:
        # Apply activation element-wise
        activation_fn = {
            "relu": relu,
            "sigmoid": sigmoid,
            "tanh": tanh
        }.get(activation, lambda x: x)  # Default to identity function
        return [activation_fn(value) for value in values]

# Load model data from JSON
with open('model.json', 'r') as json_file:
    model_data = json.load(json_file)

# Neuron function
def neuron(weights, bias, inputs):
    """
    Calculate the weighted sum of inputs and bias for a single neuron.
    """
    return sum(w * i for w, i in zip(weights, inputs)) + bias

# Forward pass through the network
def forward_pass(model_data, inputs):
    """
    Perform a forward pass through the network using the model data from JSON.
    """
    current_inputs = inputs
    for layer_name, layer_data in model_data.items():
        # Compute outputs for all neurons in the current layer
        layer_outputs = [
            neuron(weights, bias, current_inputs)
            for weights, bias in zip(layer_data["weights"], layer_data["biases"])
        ]

        # Apply the activation function to the entire layer's output
        activation = layer_data["activation"]
        current_inputs = apply_activation(activation, layer_outputs)

    return current_inputs

# Example input data
input_data = [0.5, 0.6]  # Adjust based on the input layer size

# Perform a forward pass and print the result
output = forward_pass(model_data, input_data)
print("Network Output:", output)


Network Output: [0.09341424382860124, 0.10371213700783133, 0.10131805130753267, 0.09368471996196068, 0.09239848420198778, 0.10041246509774145, 0.10744322289471599, 0.09729294397245611, 0.10447055819508498, 0.1058531735320877]


In [13]:
import math

# Activation functions
def relu(x):
    return max(0, x)

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def tanh(x):
    return math.tanh(x)

# Neuron class: Represents each individual neuron
class Neuron:
    def __init__(self, neuron_id, weights, bias, activation):
        self.neuron_id = neuron_id  # Unique ID for this neuron
        self.weights = weights
        self.bias = bias
        self.activation = activation
        self.activated_by = []  # List of neuron IDs that activate this neuron
        self.activates = []     # List of neuron IDs that this neuron activates

    def activate(self, inputs):
        """
        Activate the neuron based on its inputs, weights, and bias.
        Apply the specified activation function to the weighted sum.
        """
        z = sum(w * i for w, i in zip(self.weights, inputs)) + self.bias
        return self.apply_activation(z)

    def apply_activation(self, z):
        """
        Apply the activation function to the input value (z).
        """
        if self.activation == "relu":
            return relu(z)
        elif self.activation == "sigmoid":
            return sigmoid(z)
        elif self.activation == "tanh":
            return tanh(z)
        else:
            return z  # Identity function if no activation is defined

# Layer class: Represents each layer (set of neurons)
class Layer:
    def __init__(self, neurons):
        self.neurons = neurons  # List of neurons in this layer

    def forward(self, inputs):
        """
        Perform forward propagation through this layer.
        """
        return [neuron.activate(inputs) for neuron in self.neurons]

# Build layers from model data and set relationships
def build_layer(layer_data, neuron_start_id, previous_layer=None):
    """
    Construct a Layer object from the model data for a specific layer.
    Establish relationships (activated_by and activates) between neurons in
    the current and previous layers.
    """
    neurons = []
    for idx, (weights, bias) in enumerate(zip(layer_data["weights"], layer_data["biases"])):
        neuron_id = neuron_start_id + idx
        activation = layer_data["activation"]
        neuron = Neuron(neuron_id, weights, bias, activation)
        neurons.append(neuron)

    # If there is a previous layer, establish relationships
    if previous_layer:
        for prev_neuron in previous_layer.neurons:
            for curr_neuron in neurons:
                prev_neuron.activates.append(curr_neuron.neuron_id)
                curr_neuron.activated_by.append(prev_neuron.neuron_id)

    return Layer(neurons), neuron_start_id + len(neurons)

# Example model data
model_data = {
    "layer_0": {
        "weights": [[0.1, 0.2], [0.3, 0.4]],  # 2 neurons with 2 inputs
        "biases": [0.5, 0.6],
        "activation": "relu"
    },
    "layer_1": {
        "weights": [[0.7, 0.8], [0.9, 1.0]],  # 2 neurons with 2 inputs
        "biases": [0.1, 0.2],
        "activation": "sigmoid"
    }
}

# Build individual layers from model data
layers = []
neuron_start_id = 0
previous_layer = None
for layer_name, layer_data in model_data.items():
    layer, neuron_start_id = build_layer(layer_data, neuron_start_id, previous_layer)
    layers.append(layer)
    previous_layer = layer  # Update the previous layer

# Define standalone functions for each layer
def layer_0(inputs):
    return layers[0].forward(inputs)

def layer_1(inputs):
    return layers[1].forward(inputs)

# Perform prediction
input_data = [0.5, 0.6]
output_layer_0 = layer_0(input_data)
output_layer_1 = layer_1(output_layer_0)
predicted_class = output_layer_1.index(max(output_layer_1))

# Output results
print(f"Input to the network: {input_data}")
print(f"Output from layer 0: {output_layer_0}")
print(f"Output from layer 1: {output_layer_1}")
print(f"Predicted class: {predicted_class}")

# Print neuron relationships
print("\nNeuron Relationships:")
for layer in layers:
    for neuron in layer.neurons:
        print(f"Neuron ID: {neuron.neuron_id}")
        print(f"  Weights: {neuron.weights}")
        print(f"  Bias: {neuron.bias}")
        print(f"  Activation: {neuron.activation}")
        print(f"  Activated by: {neuron.activated_by}")
        print(f"  Activates: {neuron.activates}")
        print()


Input to the network: [0.5, 0.6]
Output from layer 0: [0.6699999999999999, 0.99]
Output from layer 1: [0.7959221758490919, 0.857294691580815]
Predicted class: 1

Neuron Relationships:
Neuron ID: 0
  Weights: [0.1, 0.2]
  Bias: 0.5
  Activation: relu
  Activated by: []
  Activates: [2, 3]

Neuron ID: 1
  Weights: [0.3, 0.4]
  Bias: 0.6
  Activation: relu
  Activated by: []
  Activates: [2, 3]

Neuron ID: 2
  Weights: [0.7, 0.8]
  Bias: 0.1
  Activation: sigmoid
  Activated by: [0, 1]
  Activates: []

Neuron ID: 3
  Weights: [0.9, 1.0]
  Bias: 0.2
  Activation: sigmoid
  Activated by: [0, 1]
  Activates: []

